<a href="https://colab.research.google.com/github/lettymoon/FATECSP-PLN/blob/main/corte_barras_pulp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho de Pesquisa Operacional

## Aplicação de Programação Linear em Problema de Corte de Barras

Aluno: Letícia Helena Cândido

Disciplina: Pesquisa Operacional

Data: 11/10/2025


# Python e Programação Linear

Python oferece diversas bibliotecas para resolver problemas de Pesquisa Operacional,
como SciPy, PuLP, Pyomo e OR-Tools.

Neste trabalho é utilizada a biblioteca **PuLP**, que permite modelar problemas
de Programação Linear e Inteira de forma simples, com sintaxe próxima da notação
matemática e com um solver open source (CBC) disponível por padrão.


# Introdução

Neste notebook é apresentado um estudo de aplicação de Programação Linear Inteira
em um problema clássico de corte de barras.

O objetivo é determinar como cortar barras de 6 metros de comprimento para atender
demandas específicas de barras menores, minimizando as perdas de material.


# Python e Programação Linear

Python oferece diversas bibliotecas para resolver problemas de Pesquisa Operacional,
como SciPy, PuLP, Pyomo e OR-Tools.

Neste trabalho é utilizada a biblioteca **PuLP**, que permite modelar problemas
de Programação Linear e Inteira de forma simples, com sintaxe próxima da notação
matemática e com um solver open source (CBC) disponível por padrão.


In [16]:
# @title Importação de bibliotecas e definição dos dados

from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus

# Comprimento da barra original (em metros)
L = 6

# Demandas de barras menores
demand_2 = 50  # barras de 2 m
demand_3 = 60  # barras de 3 m
demand_4 = 90  # barras de 4 m

print("Comprimento da barra original:", L, "m")
print("Demanda 2 m:", demand_2)
print("Demanda 3 m:", demand_3)
print("Demanda 4 m:", demand_4)


Comprimento da barra original: 6 m
Demanda 2 m: 50
Demanda 3 m: 60
Demanda 4 m: 90


#Padrões de corte considerados

Para formular o problema como um modelo de Programação Linear Inteira, definimos
alguns padrões de corte possíveis para a barra de 6 m.

Cada padrão indica quantas barras de 2 m, 3 m e 4 m serão obtidas a partir de
uma única barra de 6 m. Também calculamos a sobra (perda) associada a cada padrão.


In [17]:
# @title Definição dos padrões de corte e sobras

# Cada padrão é uma tupla: (qtd_2m, qtd_3m, qtd_4m)
patterns = {
    "padrao_2_1x2": (1, 0, 0),          # 1 barra de 2 m  + 4 m de sobra
    "padrao_2_2x2": (2, 0, 0),          # 2 barras de 2 m + 2 m de sobra
    "padrao_2_3x2": (3, 0, 0),          # 3 barras de 2 m (sem sobra)
    "padrao_3_1x3": (0, 1, 0),          # 1 barra de 3 m  + 3 m de sobra
    "padrao_23_1x2_1x3": (1, 1, 0),     # 1 de 2 m + 1 de 3 m + 1 m de sobra
    "padrao_3_2x3": (0, 2, 0),          # 2 barras de 3 m (sem sobra)
    "padrao_4_1x4": (0, 0, 1),          # 1 barra de 4 m + 2 m de sobra
    "padrao_24_1x2_1x4": (1, 0, 1),     # 1 de 2 m + 1 de 4 m (sem sobra)
}

# Calcula a sobra (perda) de cada padrão
waste_per_pattern = {}
for name, (q2, q3, q4) in patterns.items():
    used_length = 2 * q2 + 3 * q3 + 4 * q4
    waste_per_pattern[name] = L - used_length

# Mostra os padrões e suas sobras
for name, (q2, q3, q4) in patterns.items():
    print(
        f"{name}: 2m={q2}, 3m={q3}, 4m={q4} -> "
        f"sobra = {waste_per_pattern[name]} m"
    )


padrao_2_1x2: 2m=1, 3m=0, 4m=0 -> sobra = 4 m
padrao_2_2x2: 2m=2, 3m=0, 4m=0 -> sobra = 2 m
padrao_2_3x2: 2m=3, 3m=0, 4m=0 -> sobra = 0 m
padrao_3_1x3: 2m=0, 3m=1, 4m=0 -> sobra = 3 m
padrao_23_1x2_1x3: 2m=1, 3m=1, 4m=0 -> sobra = 1 m
padrao_3_2x3: 2m=0, 3m=2, 4m=0 -> sobra = 0 m
padrao_4_1x4: 2m=0, 3m=0, 4m=1 -> sobra = 2 m
padrao_24_1x2_1x4: 2m=1, 3m=0, 4m=1 -> sobra = 0 m


# Formulação do modelo

Seja $P$ o conjunto de padrões de corte considerados e, para cada padrão $p \in P$:

- $x_p$: número de barras de 6 m cortadas de acordo com o padrão $p$;
- $w_p$: sobra (em metros) de uma barra de 6 m quando se usa o padrão $p$;
- $a_{2p}$: quantidade de barras de 2 m obtidas no padrão $p$;
- $a_{3p}$: quantidade de barras de 3 m obtidas no padrão $p$;
- $a_{4p}$: quantidade de barras de 4 m obtidas no padrão $p$.

As demandas mínimas são:
- 50 barras de 2 m,
- 60 barras de 3 m,
- 90 barras de 4 m.

**Função objetivo**

Minimizar a sobra total de material:

$$
\min \sum_{p \in P} w_p \, x_p
$$

**Restrições de demanda**

- Barras de 2 m:

$$
\sum_{p \in P} a_{2p} \, x_p \ge 50
$$

- Barras de 3 m:

$$
\sum_{p \in P} a_{3p} \, x_p \ge 60
$$

- Barras de 4 m:

$$
\sum_{p \in P} a_{4p} \, x_p \ge 90
$$

Além disso, as variáveis de decisão são inteiras e não negativas:

$$
x_p \in \mathbb{Z}_{+}, \quad \forall p \in P.
$$


In [18]:
# @title Construção e resolução do modelo com PuLP

# Criação do modelo de Programação Linear Inteira
model = LpProblem("Corte_de_Barras", LpMinimize)

# Variáveis de decisão: número de barras de 6 m cortadas segundo cada padrão
x = {
    name: LpVariable(name, lowBound=0, cat="Integer")
    for name in patterns.keys()
}

# Função objetivo: minimizar a perda total
model += lpSum(waste_per_pattern[name] * x[name] for name in patterns.keys()), "Perda_total"

# Restrições de demanda
model += (
    lpSum(patterns[name][0] * x[name] for name in patterns.keys()) >= demand_2,
    "Demanda_2m",
)

model += (
    lpSum(patterns[name][1] * x[name] for name in patterns.keys()) >= demand_3,
    "Demanda_3m",
)

model += (
    lpSum(patterns[name][2] * x[name] for name in patterns.keys()) >= demand_4,
    "Demanda_4m",
)

# Resolver o modelo
model.solve()

print("Status da solução:", LpStatus[model.status])


Status da solução: Optimal


In [19]:
# @title Análise dos resultados

total_bars = 0
total_waste = 0.0

print("\nPadrões usados:")
for name in patterns.keys():
    qty = x[name].value()

    if qty is not None and qty > 0:
        q2, q3, q4 = patterns[name]
        waste = waste_per_pattern[name]
        total_bars += qty
        total_waste += waste * qty

        print(
            f"{name}: {qty:.0f} barra(s) de 6 m -> "
            f"{q2}x2m, {q3}x3m, {q4}x4m (sobra {waste} m por barra)"
        )

# Quantidades totais produzidas
produced_2 = sum(patterns[name][0] * x[name].value() for name in patterns.keys())
produced_3 = sum(patterns[name][1] * x[name].value() for name in patterns.keys())
produced_4 = sum(patterns[name][2] * x[name].value() for name in patterns.keys())

print("\nResumo:")
print(f"Total de barras de 6 m utilizadas: {total_bars:.0f}")
print(f"Sobra total (perda): {total_waste:.2f} m")
print(f"Barras de 2 m produzidas: {produced_2:.0f} (demanda mínima: {demand_2})")
print(f"Barras de 3 m produzidas: {produced_3:.0f} (demanda mínima: {demand_3})")
print(f"Barras de 4 m produzidas: {produced_4:.0f} (demanda mínima: {demand_4})")



Padrões usados:
padrao_3_2x3: 30 barra(s) de 6 m -> 0x2m, 2x3m, 0x4m (sobra 0 m por barra)
padrao_24_1x2_1x4: 90 barra(s) de 6 m -> 1x2m, 0x3m, 1x4m (sobra 0 m por barra)

Resumo:
Total de barras de 6 m utilizadas: 120
Sobra total (perda): 0.00 m
Barras de 2 m produzidas: 90 (demanda mínima: 50)
Barras de 3 m produzidas: 60 (demanda mínima: 60)
Barras de 4 m produzidas: 90 (demanda mínima: 90)


#Conclusão

O modelo de Programação Linear Inteira implementado em Python com a biblioteca PuLP
permite determinar uma combinação de padrões de corte que atende às demandas de
barras de 2 m, 3 m e 4 m, minimizando a sobra total de material.

Dessa forma, o uso de ferramentas de Pesquisa Operacional em Python mostra-se
adequado para apoiar decisões de planejamento de produção e uso eficiente de
recursos.
